TEST TRAINING WITH ADANIENT DATASET

In [193]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import ta
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError

In [ ]:

working_dir = os.getcwd()

# Construct the path to the 'data' directory relative to the working directory
folder_path = os.path.join(working_dir, "data")
df = pd.read_csv(os.path.join(folder_path, "updated_ADANIENT.csv"))
df = df.drop(["Ticker"], axis=1)
df['SMA_14'] = df['Close'].rolling(window=14).mean()
df.loc[0:13, 'SMA_14'] = df.loc[13, 'SMA_14']
df['EMA_14'] = df['Close'].ewm(span=14, adjust=False).mean()
df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
df.loc[0:13, 'RSI'] = df.loc[13, 'RSI']
df['Upper_Band'] = df['SMA_14'] + (df['Close'].rolling(window=14).std() * 2)
df['Lower_Band'] = df['SMA_14'] - (df['Close'].rolling(window=14).std() * 2)
df = df[13::]
df.head(30)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['Close'], label='Close Price')
plt.title('Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['Volume'], label='Volume', color='orange')
plt.title('Trading Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.legend()
plt.show()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['RSI'], label='RSI Price')
plt.title('RSI Over Time')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.show()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['Upper_Band'], label='Upper Band')
plt.plot(df['Date'], df['Lower_Band'], label='Lower Band')
plt.title('UB vs LB Over Time')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.show()

In [199]:
df = df.drop(['Date'], axis=1)
X = df[['Open', 'High', 'Low', 'Volume']]
y = df['Close']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.8, random_state=101)

In [204]:
sequence_model = Sequential()
sequence_model.add(Dense(50,activation="relu"))
sequence_model.add(Dense(50,activation="relu"))
sequence_model.add(Dense(25,activation="relu"))
sequence_model.add(Dense(1))

sequence_model.compile(optimizer="adam",loss=MeanSquaredError())

In [ ]:
sequence_model.fit(x=X_train,y=y_train,epochs=300,verbose=2)

In [ ]:
loss_df = pd.DataFrame(sequence_model.history.history)

loss_df.plot()

In [ ]:
sequence_model.evaluate(X_train,y_train,verbose=0)